In [1]:
import sys
sys.path.append('/cmlscratch/nehamk/Segment-Everything-Everywhere-All-At-Once')

from seem_module.modeling.BaseModel import BaseModel
from seem_module.modeling import build_model
from seem_module.utils.arguments import load_opt_from_config_files
from seem_module.utils.constants import COCO_PANOPTIC_CLASSES
from seem_module.utils.distributed import init_distributed
from seem_module.utils.visualizer import Visualizer


import os
import torch 
from detectron2.utils.colormap import random_color
from detectron2.data import MetadataCatalog
from detectron2.structures import BitMasks
from PIL import Image
from torchvision import transforms 
import numpy as np 


/cmlscratch/nehamk/miniconda3/envs/work/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Deformable Transformer Encoder is not available.


In [2]:
conf_files = 'configs/seem/focall_unicl_lang_demo.yaml'
opt = load_opt_from_config_files([conf_files])
opt = init_distributed(opt)

# META DATA
cur_model = 'None'
if 'focalt' in conf_files:
    pretrained_pth = os.path.join("seem_focalt_v0.pt")
    if not os.path.exists(pretrained_pth):
        os.system("wget {}".format("https://huggingface.co/xdecoder/SEEM/resolve/main/seem_focalt_v0.pt"))
    cur_model = 'Focal-T'
elif 'focal' in conf_files:
    pretrained_pth = os.path.join("seem_focall_v0.pt")
    if not os.path.exists(pretrained_pth):
        os.system("wget {}".format("https://huggingface.co/xdecoder/SEEM/resolve/main/seem_focall_v0.pt"))
    cur_model = 'Focal-L'

'''
build model
'''
model = BaseModel(opt, build_model(opt)).from_pretrained(pretrained_pth).eval().cuda()
with torch.no_grad():
    # model.model.sem_seg_head.predictor.lang_encoder.get_text_embeddings(COCO_PANOPTIC_CLASSES + ["background"], is_eval=True)

    thing_colors = [random_color(rgb=True, maximum=255).astype(np.int32).tolist() for _ in range(len(COCO_PANOPTIC_CLASSES))]
    thing_dataset_id_to_contiguous_id = {x:x for x in range(len(COCO_PANOPTIC_CLASSES))}
    
    MetadataCatalog.get("demo").set(
        thing_colors=thing_colors,
        thing_classes=COCO_PANOPTIC_CLASSES,
        thing_dataset_id_to_contiguous_id=thing_dataset_id_to_contiguous_id,
    )
    model.model.sem_seg_head.predictor.lang_encoder.get_text_embeddings(COCO_PANOPTIC_CLASSES + ["background"], is_eval=True)
    metadata = MetadataCatalog.get('demo')
    model.model.metadata = metadata
    model.model.sem_seg_head.num_classes = len(COCO_PANOPTIC_CLASSES)

$UNUSED$ criterion.empty_weight, Ckpt Shape: torch.Size([134])


In [17]:
# t = []
# t.append(transforms.Resize((512, 512), interpolation=Image.BICUBIC))
# transform = transforms.Compose(t)

transform = transforms.Compose([
                transforms.Resize((512, 512), interpolation=Image.BICUBIC),
                transforms.PILToTensor()
            ])

# image_ori = Image.open('person_with_coffee.jpeg').convert("RGB")
image_ori = Image.open('/fs/cml-datasets/coco/images/train2017/000000000009.jpg').convert("RGB")

width = image_ori.size[0]
height = image_ori.size[1]
images = transform(image_ori)
# image = np.asarray(image)
print(image.shape)
image_ori = np.asarray(image_ori)
# images = torch.from_numpy(image.copy()).permute(2,0,1).cuda()

batch_inputs = [{'image': images, 'height': height, 'width': width}]


with torch.no_grad():
    outputs = model.forward(batch_inputs)

(512, 512, 3)


In [18]:
images.shape

torch.Size([3, 512, 512])

In [19]:

visual = Visualizer(image_ori, metadata=metadata)

inst_seg = outputs[-1]['instances']
inst_seg.pred_masks = inst_seg.pred_masks.cpu()
inst_seg.pred_boxes = BitMasks(inst_seg.pred_masks > 0).get_bounding_boxes()
demo = visual.draw_instance_predictions(inst_seg) # rgb Image

demo.save('inst.png')


In [28]:
inst_seg

Instances(num_instances=100, image_height=404, image_width=715, fields=[pred_masks: tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        ...,

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0.

In [22]:
model.model.sem_seg_head

XdecoderHead(
  (pixel_decoder): TransformerEncoderPixelDecoder(
    (adapter_1): Conv2d(
      192, 512, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): GroupNorm(32, 512, eps=1e-05, affine=True)
    )
    (layer_1): Conv2d(
      512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): GroupNorm(32, 512, eps=1e-05, affine=True)
    )
    (adapter_2): Conv2d(
      384, 512, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): GroupNorm(32, 512, eps=1e-05, affine=True)
    )
    (layer_2): Conv2d(
      512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): GroupNorm(32, 512, eps=1e-05, affine=True)
    )
    (adapter_3): Conv2d(
      768, 512, kernel_size=(1, 1), stride=(1, 1), bias=False
      (norm): GroupNorm(32, 512, eps=1e-05, affine=True)
    )
    (layer_3): Conv2d(
      512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
      (norm): GroupNorm(32, 512, eps=1e-05, affine=True)